# Inplementation of Internal State-based Uncertainty Estimation

1. 数据集预处理：将不同格式的数据集处理成input+gt的格式，方便判断模型的correctness，这一部分的采用固定不可调整的prompt，即Context: Question: Options: Answer:格式
2. 生成回复，为每个模型确定一个prompt，一个max_new_tokens数，然后生成回复
3. 计算回复部分的correctness指标，判断模型的回复是否正确
4. 计算uncertainty指标，包括PE, LN-PE, SAR, Ours
5. 计算AUROC，绘制AUROC/Correctness-Threshold曲线

In [34]:
from utils import *

datasets.disable_caching()
torch.set_grad_enabled(False)
# print_sys_info()

In [25]:
# Eval Result Config
model_names = [
    "vicuna-7b-v1.1",
    "vicuna-13b-v1.1",
    "vicuna-33b-v1.3",
]
dst_names = [
    "sciq",
    "coqa",
    "triviaqa",
    "medmcqa",
    "MedQA-USMLE-4-options",
]

c_metrics = [
    'rougel'
    'sentsim'
    'include'
]

dst_types = [
    "long",
    "short"
]

acc_map={
    "vicuna-7b-v1.1": {
        "sciq" : 0.65,
        "coqa" : 0.0,
        "triviaqa" : 0.55,
        "medmcqa" : 0.0,
        "MedQA-USMLE-4-options" : 0.0
    },
    "vicuna-13b-v1.1": {
        "sciq" : 0.0,
        "coqa" : 0.0,
        "triviaqa" : 0.0,
        "medmcqa" : 0.0,
        "MedQA-USMLE-4-options" : 0.0
    },
    "vicuna-33b-v1.3": {
        "sciq" : 0.0,
        "coqa" : 0.0,
        "triviaqa" : 0.0,
        "medmcqa" : 0.0,
        "MedQA-USMLE-4-options" : 0.0
    }
}

cth_map = dict(
    rougel=dict(
        coqa_long=1
    )
)

def get_eval_main_result_path(model_name, dst_name, dst_type):
    return f"eval_results/{model_name}/{dst_name}_{dst_type}"

def get_eval_cross_result_path(model_name, train_dst_name, train_dst_type, test_dst_name, test_dst_type):
    return f"cross_eval_results/{model_name}/rougel/v_c_{train_dst_name}_{train_dst_type}_mean_soft_best.pth/{test_dst_name}_{test_dst_type}"

In [24]:
test_dst = Dataset.load_from_disk(get_eval_main_result_path('vicuna-7b-v1.1', 'MedQA-USMLE-4-options', 'long'))
for i in range(0,30):
    for k in ['gt']:
        print(f"{k}: {test_dst[i][k]}")

gt: Tell the attending that he cannot fail to disclose this mistake
gt: Cross-linking of DNA
gt: Cholesterol embolization
gt: Lactose-fermenting, gram-negative rods forming pink colonies on MacConkey agar
gt: Ketotifen eye drops
gt: Reassurance and continuous monitoring
gt: Common iliac artery aneurysm
gt: Clopidogrel
gt: Active or recurrent pelvic inflammatory disease (PID)
gt: Silvery plaques on extensor surfaces
gt: It is an HIV-1/HIV2 antibody differentiation immunoassay
gt: Ruxolitinib
gt: Meningioma
gt: A reduction in diastolic filling time
gt: Rotavirus
gt: Gallbladder cancer
gt: IL-4
gt: Matching
gt: Ibuprofen + colchicine +/- omeprazole
gt: Benzodiazepine intoxication
"
gt: Previous radiation therapy
gt: 22q11 deletion
gt: Histoplasma capsulatum infection
gt: Staphylococcus aureus
gt: Intubate with mechanical ventilation
gt: Respiratory burst
gt: Steeple sign
gt: Induction of CYP3A4 by rifampin leading to decreased serum levels of ethinylestradiol and progesterone
gt: Increase

In [ ]:
# # Model Config
# model_name = "vicuna-7b-v1.1"
# hooked_transformer_name = "llama-7b-hf"
# hf_model_path = os.path.join(os.environ["my_models_dir"], model_name)
# hf_tokenizer = AutoTokenizer.from_pretrained(hf_model_path)
# hf_model = AutoModelForCausalLM.from_pretrained(hf_model_path)
# 
# model = HookedTransformer.from_pretrained_no_processing(hooked_transformer_name, dtype='bfloat16', hf_model=hf_model, tokenizer=hf_tokenizer, default_padding_side='left')
# 
# # Aux Models
# se_bert_name = "microsoft/deberta-large-mnli"
# nli_pipe = pipeline("text-classification", model=se_bert_name, device=0)
# 
# # sar_bert_name = 'cross-encoder/stsb-roberta-large'
# # # sar_bert_name = 'sentence-transformers/all-MiniLM-L6-v2'
# # sar_bert = SentenceTransformer(sar_bert_name)

In [ ]:
# Our Method OLD
def compute_certainty_vector_mean(train_dst, layers, act_name, batch_size=8, model):
    def get_paired_dst_sciq(train_dst):
        tmp_pos = "Question:{q} Options:{o} The correct answer is:"
        tmp_neg = "Question:{q} Options:{o} The incorrect answer is:"
    
        # sciq_train_dst = sciq_train_dst.filter(lambda x: x['rougel'] > 0.5)
    
        def get_pos_example(example):
            example['input'] = tmp_pos.format(q=example['question'], o=", ".join(example['options']))
            example['washed_output'] = f"{example['input']}{example['gt']}"
            return example
    
        def get_neg_example(example, idx):
            example['input'] = tmp_neg.format(q=example['question'], o=", ".join(example['options']))
            wrong_options = [opt for opt in example['options'] if opt != example['gt']]
            if wrong_options:
                random.seed(42 + idx)
                wrong_answer = random.choice(wrong_options)
            else:
                wrong_answer = "wrong answer"
            example['washed_output'] = f"{example['input']}{wrong_answer}"
            return example
    
        dst_pos = train_dst.map(get_pos_example, new_fingerprint=str(time()))
        dst_neg = train_dst.map(get_neg_example, with_indices=True, new_fingerprint=str(time()))
        return dst_pos, dst_neg
    
    
    def get_paired_dst_coqa(train_dst):
        def get_pos_example(example):
            example['washed_output'] = f"{example['input']}The correct answer is {example['gt']}"
            return example
    
        def get_neg_example(example, idx):
            wrong_options = [opt for opt in example['answers']['input_text'] if opt != example['gt']]
            if wrong_options:
                random.seed(42 + idx)
                wrong_answer = random.choice(wrong_options)
            else:
                wrong_answer = "wrong answer"
            example['washed_output'] = f"{example['input']}The wrong answer is {wrong_answer}"
            return example
    
        dst_pos = train_dst.map(get_pos_example, new_fingerprint=str(time()))
        dst_neg = train_dst.map(get_neg_example, with_indices=True, new_fingerprint=str(time()))
        return dst_pos, dst_neg
    
    
    def get_paired_dst_triviaqa(train_dst):
        def get_pos_example(example):
            example['washed_output'] = f"{example['input']}The correct answer is {example['gt']}"
            return example
    
        def get_neg_example(example, idx):
            next_idx = idx + 1 if idx + 1 < len(train_dst) else 0
            wrong_answer = train_dst[next_idx]['gt']
            example['washed_output'] = f"{example['input']}The wrong answer is {wrong_answer}"
            return example
    
        dst_pos = train_dst.map(get_pos_example, new_fingerprint=str(time()))
        dst_neg = train_dst.map(get_neg_example, with_indices=True, new_fingerprint=str(time()))
        return dst_pos, dst_neg
    
    
    def get_paired_dst_medmcqa(train_dst):
        def get_pos_example(example):
            example['washed_output'] = f"{example['input']}The correct answer is {example['gt']}"
            return example
    
        def get_neg_example(example, idx):
            wrong_options = [opt for opt in example['options'] if opt != example['gt']]
            if wrong_options:
                random.seed(42 + idx)
                wrong_answer = random.choice(wrong_options)
            else:
                wrong_answer = "wrong answer"
            example['washed_output'] = f"{example['input']}The wrong answer is {wrong_answer}"
            return example
    
        dst_pos = train_dst.map(get_pos_example, new_fingerprint=str(time()))
        dst_neg = train_dst.map(get_neg_example, with_indices=True, new_fingerprint=str(time()))
        return dst_pos, dst_neg
    
    func_map = {
        'allenai/sciq': get_paired_dst_sciq,
        'stanfordnlp/coqa': get_paired_dst_coqa,
        'lucadiliello/triviaqa': get_paired_dst_triviaqa,
        'openlifescienceai/medmcqa': get_paired_dst_medmcqa,
        'GBaker/MedQA-USMLE-4-options': get_paired_dst_medmcqa
    }
    func = func_map[dst_name]
    dst_pos, dst_neg = func(train_dst)
    
    data_pos = dst_pos['washed_output']
    data_neg = dst_neg['washed_output']
    data_size = len(data_pos)
    full_act_names = [utils.get_act_name(act_name, l) for l in sorted(layers)]
    v_c = torch.zeros((len(layers), 1, model.cfg.d_model)).cuda()

    for i in tqdm(range(0, data_size, batch_size)):
        batch_pos = data_pos[i:i + batch_size]
        batch_neg = data_neg[i:i + batch_size]

        _, cache_pos = model.run_with_cache(batch_pos, names_filter=lambda x: x in full_act_names, padding_side='left')  # logits: (bsz pos vocab) cache: dict
        _, cache_neg = model.run_with_cache(batch_neg, names_filter=lambda x: x in full_act_names, padding_side='left')  # logits: (bsz pos vocab) cache: dict

        cache_pos = einops.rearrange([cache_pos[name] for name in full_act_names], 'l b p d -> b l p d')
        cache_neg = einops.rearrange([cache_neg[name] for name in full_act_names], 'l b p d -> b l p d')

        cache_pos = cache_pos[:, :, [-1], :]
        cache_neg = cache_neg[:, :, [-1], :]

        v_c += (cache_pos.sum(dim=0) - cache_neg.sum(dim=0))

    v_c /= data_size

    v_c = v_c.cpu().float()
    v_c = F.normalize(v_c, p=2, dim=-1)
    return v_c

# clean_exp exp
def clean_exp(dst, v_c, layers, act_name):
    fig = go.Figure()
    c_scores = []
    w_scores = []
    labels = []
    u_scores = []
    u_scores_z = []
    all_pe_u_scores = []
    all_ln_pe_u_scores = []

    def batch_get_result(examples):
        all_outputs = []
        all_num_answer_tokens = []
        all_num_input_tokens = list(map(len, model.to_str_tokens(examples['input'])))
        bsz = len(examples['input'])

        for i in range(bsz):
            example = {k: examples[k][i] for k in examples.keys()}
            if example.get("options"):
                wrong_options = [opt for opt in example['options']]
                for opt in wrong_options:
                    if opt == example['gt']:
                        wrong_options.remove(opt)
                        break
            elif example.get("answers"):
                wrong_options = [opt for opt in example['answers']['input_text']]
                for opt in wrong_options:
                    if opt == example['gt']:
                        wrong_options.remove(opt)
                        break
                wrong_options = wrong_options[:3]
            else:
                wrong_options = ['wrong answer', 'bad answer', 'incorrect answer']
            correct_output = example['input'] + example['gt']
            wrong_outputs = [example['input'] + opt for opt in wrong_options]
            all_outputs.extend([correct_output] + wrong_outputs)
            num_answer_tokens = list(map(len, model.to_str_tokens([example['gt']] + wrong_options)))
            all_num_answer_tokens.append(num_answer_tokens)

        full_act_names = [utils.get_act_name(act_name, l) for l in sorted(layers)]

        batch_logits, batch_cache = model.run_with_cache(all_outputs, names_filter=lambda x: x in full_act_names,
                                                         device='cpu',
                                                         padding_side='left')  # logits: (bsz pos vocab) cache: dict
        batch_cache = einops.rearrange([batch_cache[name] for name in full_act_names],
                                       'l b p d -> b l p d').float().cpu()
        batch_cache = einops.rearrange(batch_cache, '(b o) l p d -> b o l p d', o=4)
        batch_cache = batch_cache[:, :, :, [-1], :]

        batch_logits = batch_logits.cpu().float()
        batch_logits = einops.rearrange(batch_logits, '(b o) p v -> b o p v', o=4)

        for i, lg_4 in enumerate(batch_logits):
            num_answer_tokens = all_num_answer_tokens[i]
            num_input_tokens = all_num_input_tokens[i]
            for j, lg in enumerate(lg_4):
                output = all_outputs[i * 4 + j]
                answer_lg = lg[-num_answer_tokens[j] - 1:-1]
                answer_prob = F.softmax(answer_lg, dim=-1)
                answer_target_prob = answer_prob.max(dim=-1).values
                pe = -torch.log(answer_target_prob).sum().item()
                # print(f"pe:{pe}")
                ln_pe = -torch.log(answer_target_prob).mean().item()
                # print(f"ln_pe:{ln_pe}")
                all_pe_u_scores.append(pe)
                all_ln_pe_u_scores.append(ln_pe)

        batch_in_vivo_auroc = []
        for i in range(bsz):
            cache = batch_cache[i]
            u_score = einsum('b l p d, l p d -> b', cache, v_c)
            u_score_z = (u_score - u_score.mean()) / u_score.std()

            u_score = u_score.tolist()
            u_score_z = u_score_z.tolist()

            in_vivo_auroc = roc_auc_score([1, 0, 0, 0], u_score)
            batch_in_vivo_auroc.append(in_vivo_auroc)
            # if u_score[0] > max(u_score[1:]):
            #     batch_in_vivo_auroc.append(1)
            # else:
            #     batch_in_vivo_auroc.append(0)

            c_scores.append(u_score_z[0])
            w_scores.extend(u_score_z[1:])
            labels.extend([1, 0, 0, 0])

            # assert len(u_score) == 4, f"{len(u_score)} {example['options']}"
            u_scores.extend(u_score)
            u_scores_z.extend(u_score_z)

        examples['in_vivo_auroc'] = batch_in_vivo_auroc
        return examples

    new_dst = dst.map(batch_get_result, new_fingerprint=str(time()), batched=True, batch_size=4)

    in_vivo_auroc = sum(new_dst['in_vivo_auroc']) / len(new_dst['in_vivo_auroc'])
    flag = in_vivo_auroc > 0.5
    in_vivo_auroc = in_vivo_auroc if flag else 1 - in_vivo_auroc
    print(f"in-vivo u_score auroc: {in_vivo_auroc}")

    in_vitro_auroc = roc_auc_score(labels, u_scores)
    in_vitro_auroc = in_vitro_auroc if flag else 1 - in_vitro_auroc
    print(f"in-vitro u_score auroc: {in_vitro_auroc}")

    in_vitro_auroc_z = roc_auc_score(labels, u_scores_z)
    in_vitro_auroc_z = in_vitro_auroc_z if flag else 1 - in_vitro_auroc_z
    print(f"in-vitro u_score_z auroc: {in_vitro_auroc_z}")

    in_vitro_pe_auroc = roc_auc_score(labels, all_pe_u_scores)
    print(f"in-vitro pe auroc: {in_vitro_pe_auroc}")

    in_vitro_ln_pe_auroc = roc_auc_score(labels, all_ln_pe_u_scores)
    print(f"in-vitro ln_pe auroc: {in_vitro_ln_pe_auroc}")

    fig.add_trace(go.Histogram(x=c_scores, name='Correct', opacity=0.5, nbinsx=100))
    fig.add_trace(go.Histogram(x=w_scores, name='Wrong', opacity=0.5, nbinsx=100))
    fig.update_layout(barmode='overlay')
    fig.show()

In [ ]:
example = test_dst.filter(lambda x: x['rougel'] < 0.1)[1]
example = test_dst[2]
print(f"gt:{example['gt']}")
print(f"options:{example['options']}")

str_tokens = model.to_str_tokens(f":{example['washed_answer']}", prepend_bos=False)[1:]
fig = make_subplots(rows=2, cols=1, subplot_titles=("Token Level", "Sentence Level"), row_heights=[0.5, 0.5])

fig.add_trace(go.Scatter(x=list(range(len(str_tokens))), y=example['u_score_pe_all'], mode='lines+markers'), row=1, col=1)
fig.update_xaxes(title_text='Token', tickvals=list(range(len(str_tokens))), ticktext=str_tokens, row=1, col=1)

sentence_u_score_pe_all = []
indices = [0]+[i for i, x in enumerate(str_tokens) if x == '.']+[-1]
spans = [(indices[i], indices[i+1]) for i in range(len(indices)-1)]
print(len(indices))
for span in spans:
    sentence_score = sum(example['u_score_pe_all'][span[0]:span[1]]) / (span[1] - span[0])
    sentence_u_score_pe_all.extend([sentence_score] * (span[1] - span[0]))
sentence_u_score_pe_all.append(sentence_u_score_pe_all[-1])
# print(str_tokens)
for i, sentence in enumerate(example['washed_answer'].split(".")):
    print(i+1,sentence.replace("\n",' ').strip())
# print(len(example['u_score_pe_all']))
# print(sentence_u_score_pe_all)
# print(len(sentence_u_score_pe_all))

fig.add_trace(go.Scatter(x=list(range(len(str_tokens))), y=sentence_u_score_pe_all, mode='lines+markers'), row=2, col=1)
fig.update_xaxes(title_text='Sentence', tickvals=list(range(len(str_tokens))), ticktext=str_tokens, row=2, col=1)

fig.update_layout(height=1000, width=2500, margin=dict(l=0, r=0, b=50, t=50), title_text=example['washed_answer'])
fig.show()